In [29]:
import string
from collections import defaultdict
import random
import re
import numpy as np
from statistics import *
import math

## Read in Lines

In [30]:

file_path = 'cl2_dash'

In [31]:
name_file = open(file_path, "r")

FileNotFoundError: [Errno 2] No such file or directory: 'cl2_dash'

In [32]:
#line = name_file.readline().replace('\n', " ").replace('.', '').replace("'","").replace('_','').replace('\xa0', '').replace('#','').replace('�',"").lower()

In [33]:
line =  name_file.readline()

NameError: name 'name_file' is not defined

In [34]:
name_file.close()

NameError: name 'name_file' is not defined

# line = ' deadpool spider man hulk red skull iron man '

In [117]:
line = re.sub(r'[^a-zA-Z\-\'\s=]','', line).replace('\n', ' ').replace("'", " ").replace('\xa0','').lower()

In [118]:
line

' deadpool spider man hulk red skull iron man '

## Roundoff

In [119]:
def roundoff(value):
    if( (value < 1.0) and (value > 0.998) ):
        return 1.0
    else:
        return value
        

In [120]:
def get_length(inputt):
    words = [ w for w in inputt.split(' ') if w != ""]
    m = mean(map(len, words))
    s = stdev(map(len, words))
    return math.floor(random.gauss(m, s))
    

In [121]:
print(get_length(line))

5


## TWO LETTER FUNCTIONS

In [122]:
def count_ocurrance_two(inputt):
    ocurrancies = defaultdict(lambda: 0)
    
    last_char_read = None
    
    for ch in inputt:
        if (last_char_read != None):
            ocurrancies[last_char_read +  ch] += 1
        
        last_char_read = ch
    
    return ocurrancies
            

In [123]:
def create_fequencies_two(inputt):
    occurences = count_ocurrance_two(inputt)
    freq = defaultdict(lambda: 0)
    
    regular_occurance = { key:value for key, value in occurences.items() if key[1] != ' ' }
    word_ending_occurances = { key:value for key, value in occurences.items() if key[1] == ' ' }
    
    if(len(occurences) > 0):
        
        for k , v in regular_occurance.items():
            freq[k] = v / float( sum([value for key, value in regular_occurance.items() if key[0] == k[0]]))
        
        for k , v in word_ending_occurances.items():
            freq[k] = v / float( sum([value for key, value in word_ending_occurances.items()]))
    
    return freq
        
        
           
         

In [124]:
def execute_two(freq):
    last_key = None
    cum_value = 0.0
    
    for key in sorted({k for k in  freq.keys() if k[1] != ' ' }):
        if((last_key != None) and (last_key[0] != key[0])):
            cum_value = 0.0
        cum_value += freq[key]
        cum_value = roundoff(cum_value) 
        freq[key] = cum_value
        last_key = key
    
    cum_value = 0.0
    
    for key in sorted({k for k in  freq.keys() if k[1] == ' ' }):
        cum_value += freq[key]
        cum_value = roundoff(cum_value) 
        freq[key] = cum_value
    
    return freq
    
    
        

    

## Three Letter Functions

In [125]:
def count_ocurrance_three(inputt):
    occurances = defaultdict(lambda: 0)
    
    last_char = None
    next_2_last = None
    
    for ch in inputt:
        if ((last_char != None) and (next_2_last != None)):
            occurances[next_2_last + last_char + ch] += 1
        
        next_2_last = last_char
        last_char = ch
    
    return {key:value for key , value in occurances.items() if key[1] != ' '}

In [126]:
def create_fequencies_three(inputt):
    occurences =  count_ocurrance_three(inputt)
    
    freq = defaultdict(lambda: 0)
    
    regular_occurance = { key:value for key, value in occurences.items() if key[2] != ' ' }
    word_ending_occurances = { key:value for key, value in occurences.items() if key[2] == ' ' }
    
    if(len(occurences) > 0):
        
        for k , v in regular_occurance.items():
            freq[k] = v / float( sum([value for key, value in regular_occurance.items() if (key[0] == k[0]) and (key[1] == k[1])]))
        
        for k , v in word_ending_occurances.items():
            freq[k] = v / float( sum([value for key, value in word_ending_occurances.items() if k[0] == key[0]]))
    
    return freq
    
    
    

In [127]:
def execute_three(freq):
    last_key = None
    cum_value = 0.0
    
    for key in sorted({k for k in freq.keys() if k[2] != ' '}):
        if (last_key != None) and (last_key[0] != key[0]  or last_key[1] != key[1]):
            cum_value = 0.0
        
        cum_value += freq[key]
        
        cum_value = roundoff(cum_value)
        
        freq[key] = cum_value
        last_key = key
    
    last_key = None
    cum_value = 0.0
    
    for key in sorted({k for k in freq.keys() if k[2] == ' '}):
        if (last_key != None) and (last_key[0] != key[0]):
            cum_value = 0.0
        
        cum_value += freq[key]
        
        cum_value = roundoff(cum_value)
        
        freq[key] = cum_value
        last_key = key
    
    return freq
    
    

## Random Number Gen


In [128]:
trip_cum_prob = execute_three(create_fequencies_three(line))

In [129]:
pair_cum_prob = execute_two(create_fequencies_two(line))

In [130]:
def get_starting_letter():
    rand = random.random()
    
    return sorted( [ key for key, value in pair_cum_prob.items() if key[0] == " " and key[1] != " " and value >= rand])[0][1]
    
    


In [131]:
def get_next_letter(first_letter, second_letter):
    rand = random.random()
    trip_list = sorted([key for key, value in trip_cum_prob.items() if key[0] == first_letter and key[1] == second_letter and key[2] != ' ' and value >= rand])
    if(len(trip_list)):
        return trip_list[0][2]
    
    pair_list = sorted([key for key, value in pair_cum_prob.items() if key[1] == second_letter and key[0] != ' ' and value >= rand])
    if(len(pair_list)):
        return pair_list[0][1]
   
    return get_starting_letter()
        

In [132]:
def get_last_letter(current_last, sec_to_last):
    rand = random.random()
    
    trip_list = sorted([key for key, value in trip_cum_prob.items() if key[0] == current_last  and key[2] == ' ' and value >= rand])
    if(len(trip_list)):
        return trip_list[0][1]
    
    valid_letters = {k[0] for k in trip_cum_prob.keys() if k[2] == ' '}
    pair_list = sorted([key for key, value in pair_cum_prob.items() if (key[0] == current_last) and (key[1] in valid_letters) and (value >= rand)])
    if(len(pair_list)):
        new_last = pair_list[0][1]
        return new_last + get_last_letter(new_last, sec_to_last)

    return get_next_letter(current_last, sec_to_last)
    

In [133]:
def generate_name(inputt):
    name = " " + get_starting_letter()
    
    for i in range(get_length(inputt) -2):
        name += get_next_letter(name[-2], name[-1])
        
    name += get_last_letter(name[-1], name[-2])
    return name.strip().capitalize()
    

In [164]:
for i in range(50):
    print(generate_name(line))

Iron
Irono
Derol
Skulk
Iron
Reddded
Reder
Hulk
Skulllk
Man
Spided
Mannn
Dean
Hulkks
Ired
Skulk
Derol
Spon
Hulllllk
Skulk
Reddd
Mana
Red
Skulkl
Mannnnn
Skull
Man
Reded
I
Reder
Ded
Hulll
Skulll
Hulkkk
Hulk
Skull
Der
Mannnn
Deaded
I
Mannn
H
Man
Hull
Mannnn
Mannn
Skm
Mannnn
Mannn
Man


In [142]:
(line)

' deadpool spider man hulk red skull iron man '

In [143]:
create_fequencies_two(line)

defaultdict(<function __main__.create_fequencies_two.<locals>.<lambda>>,
            {' d': 0.125,
             ' h': 0.125,
             ' i': 0.125,
             ' m': 0.25,
             ' r': 0.125,
             ' s': 0.25,
             'ad': 0.3333333333333333,
             'an': 0.6666666666666666,
             'd ': 0.125,
             'de': 0.6666666666666666,
             'dp': 0.3333333333333333,
             'ea': 0.3333333333333333,
             'ed': 0.3333333333333333,
             'er': 0.3333333333333333,
             'hu': 1.0,
             'id': 0.5,
             'ir': 0.5,
             'k ': 0.125,
             'ku': 1.0,
             'l ': 0.25,
             'lk': 0.5,
             'll': 0.5,
             'ma': 1.0,
             'n ': 0.375,
             'ol': 0.3333333333333333,
             'on': 0.3333333333333333,
             'oo': 0.3333333333333333,
             'pi': 0.5,
             'po': 0.5,
             'r ': 0.125,
             're': 0.5,
             '